# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK

## Feeding an AtomsBase AbstractSystem to DFTK
In this example we construct a silicon system using the `ase.build.bulk` routine
from the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
(ASE), which is exposed by [ASEconvert](https://github.com/mfherbst/ASEconvert.jl)
as an AtomsBase `AbstractSystem`.

In [2]:
# Construct bulk system and convert to an AbstractSystem
using ASEconvert
system_ase = ase.build.bulk("Si")
system = pyconvert(AbstractSystem, system_ase)

FlexibleSystem(Si₂, AtomsBase.Periodic, box=[[0.0, 2.715, 2.715], [2.715, 0.0, 2.715], [2.715, 2.715, 0.0]]u"Å")

To use an AbstractSystem in DFTK, we attach pseudopotentials, construct a DFT model,
discretise and solve:

In [3]:
system = attach_psp(system; Si="hgh/lda/si-q4")

model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.921680490280                   -0.69    5.8
  2   -7.926165525190       -2.35       -1.22    1.0
  3   -7.926837884952       -3.17       -2.37    1.6
  4   -7.926861463998       -4.63       -3.05    2.6
  5   -7.926861652517       -6.72       -3.46    1.9
  6   -7.926861673195       -7.68       -3.90    2.0
  7   -7.926861679913       -8.17       -4.22    1.5
  8   -7.926861681632       -8.76       -4.68    1.5
  9   -7.926861681859       -9.64       -5.28    2.0
 10   -7.926861681872      -10.91       -5.94    2.0
 11   -7.926861681873      -11.96       -6.76    2.0
 12   -7.926861681873      -13.56       -7.73    2.1
 13   -7.926861681873      -14.75       -7.98    3.5
 14   -7.926861681873   +    -Inf       -8.71    1.8


If we did not want to use ASE we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

In [4]:
using AtomsIO

# Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),

which directly yields an AbstractSystem.

In [5]:
system = load_system("Si.extxyz")

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.921695339181                   -0.69    5.8
  2   -7.926164417156       -2.35       -1.22    1.0
  3   -7.926838349642       -3.17       -2.37    1.9
  4   -7.926861516970       -4.64       -3.06    2.6
  5   -7.926861652811       -6.87       -3.46    2.0
  6   -7.926861673244       -7.69       -3.89    1.5
  7   -7.926861680321       -8.15       -4.29    1.5
  8   -7.926861681723       -8.85       -4.80    1.6
  9   -7.926861681859       -9.87       -5.27    1.9
 10   -7.926861681871      -10.90       -5.82    1.8
 11   -7.926861681873      -11.85       -6.61    1.9
 12   -7.926861681873      -13.20       -7.78    2.2
 13   -7.926861681873   +  -15.05       -8.16    4.1


The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [6]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.921693875074                   -0.69    5.8
  2   -7.926166622817       -2.35       -1.22    1.0
  3   -7.926840394110       -3.17       -2.37    1.8
  4   -7.926864926139       -4.61       -3.03    2.9
  5   -7.926865050795       -6.90       -3.38    1.8
  6   -7.926865079767       -7.54       -3.76    1.9
  7   -7.926865091721       -7.92       -4.37    1.4


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [7]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, AtomsBase.Periodic, box=[[0.0, 5.13, 5.13], [5.13, 0.0, 5.13], [5.13, 5.13, 0.0]]u"a₀")

Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [8]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, AtomsBase.Periodic, box=[[0.0, 5.131551291421279, 5.131551291421279], [5.131551291421279, 0.0, 5.131551291421279], [5.131551291421279, 5.131551291421279, 0.0]]u"a₀")